In [ ]:
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from emnist import extract_training_samples
from tensorflow.keras import *
from tensorflow.keras.layers import *
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import sys
import os
import json
from sklearn.utils import resample

sys.path.insert(1, os.path.join(sys.path[0], '../../..'))

from utils import *
# tf.compat.v1.enable_eager_execution()
%matplotlib inline
%config Completer.use_jedi = False

tf.config.list_physical_devices('GPU')

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
show_dataset(x_train)

In [ ]:
num_classes = 10 

mean = np.mean(x_train,axis=(0, 1, 2, 3))
std = np.std(x_train, axis=(0, 1, 2, 3))

x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

y_train = utils.to_categorical(y_train, num_classes) 
y_test = utils.to_categorical(y_test, num_classes) 

In [ ]:
x_train.shape

In [ ]:
results = {}

# Bagging BENN

In [ ]:
import tensorflow as tf
import larq as lq

kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip",
              use_bias=False)
 
def get_model():    
    model = tf.keras.models.Sequential([
        lq.layers.QuantConv2D(32, 3,
                              kernel_quantizer="ste_sign",
                              kernel_constraint="weight_clip",
                              use_bias=False,
                              input_shape=(32, 32, 3)),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),

        lq.layers.QuantConv2D(32, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    #     tf.keras.layers.Dropout(0.4),

        lq.layers.QuantConv2D(64, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),

        lq.layers.QuantConv2D(64, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    #     tf.keras.layers.Dropout(0.4),

        lq.layers.QuantConv2D(128, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),

        lq.layers.QuantConv2D(128, 3, padding="same", **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
    #     tf.keras.layers.Dropout(0.4),


        tf.keras.layers.Flatten(),

        lq.layers.QuantDense(128, **kwargs),
        tf.keras.layers.Activation("elu"),
        tf.keras.layers.BatchNormalization(),


        lq.layers.QuantDense(10, **kwargs),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation("softmax")
    ])
    
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
members = []
n = len(x_train)

In [ ]:
# for i in range(8):
#     model = get_model()
#     idx = resample(range(n), replace=False, n_samples=n*2//3)
#     model.fit(x_train[idx], y_train[idx], batch_size=256, epochs=100,
#               validation_data=(x_test, y_test))
#     members.append(model)

In [ ]:
# for i, model in enumerate(members):
#     model.save('vanilla_cifar10_bnn_{}.h5'.format(i+1))

In [ ]:
for i in range(8):
    members.append(tf.keras.models.load_model('vanilla_cifar10_bnn_{}.h5'.format(i+1)))

In [ ]:
len(members)

In [ ]:
def evaluate_ensamble(ensamble, x_test, y_test):
    y_pred = [model.predict_proba(x_test) for model in ensamble]
    a = np.mean(y_pred, axis=0)
    return full_evaluate2(pred=a, y=y_test)

In [ ]:
def measure_ensamble_size(ensamble):
    return sum([float(lq.models.ModelProfile(model).generate_table()[-1][-3]) for model in ensamble])

In [ ]:
evaluate_ensamble(members, x_test, y_test)

In [ ]:
measure_ensamble_size(members)

In [ ]:
results = {}

In [ ]:
results['base'] = evaluate_ensamble(members, x_test, y_test)
results['base']['size (kb)'] = measure_ensamble_size(members)

In [ ]:
with open('results_bnn_bagging.json', 'r', encoding ='utf8') as json_file:
    results = json.load(json_file)

In [ ]:
results

# BNN + prunning

In [ ]:
def evaluate_ensamble_pruned(ensamble, x_test, y_test):
    y_pred = [model.predict_on_batch(x_test) for model in ensamble]
    a = np.mean(y_pred, axis=0)
    return full_evaluate2(pred=a, y=y_test)

In [ ]:
for i in range(8):
    try:
        for j, model in enumerate(members):
            members[j] = prune_model(model, 0.1, opt='adam', method='l1')
    except:
        break
    for model in members:
        idx = resample(range(n), replace=False, n_samples=n*2//3)
        model.fit(x_train[idx], y_train[idx], batch_size=256, epochs=15,
              validation_data=(x_test, y_test))
    results['bnn_node_pruning_{}'.format((0.9)**(i+1))] = evaluate_ensamble_pruned(members, x_test, y_test)
    results['bnn_node_pruning_{}'.format((0.9)**(i+1))]['size (kb)'] = measure_ensamble_size(members)
    with open('results_bnn_bagging.json', 'w', encoding ='utf8') as json_file:
        json.dump(results, json_file, ensure_ascii = True)